<a href="https://colab.research.google.com/github/fionatjahjono/ml_project/blob/main/recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import gensim.downloader

In [2]:
df_user = pd.read_csv("/content/user_table.csv", index_col=0)
df_user = df_user.drop("user_name", axis=1)

df_user

,user_interest,user_past_event,latitude,longitude,user_split
user_id,,,,,
1,"2, 1","5, 1",-6.333183,106.774398,train
2,"1, 5","1, 6",-6.274905,106.821413,train
3,"3, 6","6, 1",-6.114966,106.839932,train
4,"2, 6","6, 2",-6.138156,106.698396,train
5,"1, 2","1, 5",-6.281317,106.688195,train
...,...,...,...,...,...
196,"4, 2","2, 1",-6.195217,106.586729,test
197,"2, 5","6, 3",-6.284183,106.913469,test
198,"2, 2","5, 2",-6.234056,106.682485,test


In [3]:
df_user[['user_interest', 'user_past_event']] = df_user[['user_interest', 'user_past_event']].apply(lambda x: x.str.split(', '))
df_user_expanded = df_user.explode('user_interest').explode('user_past_event')
#df_user_expanded.reset_index(inplace=True)
df_user_expanded['row_number'] = range(1, len(df_user_expanded) + 1)

df_user_expanded

,user_interest,user_past_event,latitude,longitude,user_split,row_number
user_id,,,,,,
1,2,5,-6.333183,106.774398,train,1
1,2,1,-6.333183,106.774398,train,2
1,1,5,-6.333183,106.774398,train,3
1,1,1,-6.333183,106.774398,train,4
2,1,1,-6.274905,106.821413,train,5
...,...,...,...,...,...,...
199,1,1,-6.284079,106.596490,test,796
200,6,5,-6.310049,106.982598,test,797
200,6,4,-6.310049,106.982598,test,798


In [4]:
df_event = pd.read_csv("/content/event_table.csv", index_col=0)
df_event = df_event.drop("index_to_drop", axis=1)
df_event = df_event.drop("date", axis=1)
df_event = df_event.drop("event_price", axis=1)
df_event = df_event.drop("split", axis=1)

df_event

,city_id,event_name,category_id,event_description,latitude,longitude
event_id,,,,,,
111,1,Jakarta Bird Land Ancol,1,Jakarta Bird Land adalah taman bermain sekalig...,-61266245,1068431539
122,1,Magic Art 3D Museum Jakarta,2,Step into a realm where art comes to life at t...,-61359480,1068121314
113,1,Sea World Ancol,1,Dive into the depths of the ocean without leav...,-61257784,1068428327
114,1,Samudra Ancol,1,Experience the magic of marine life with a vis...,-61250180,1068436004
115,1,Dufan Ancol,1,"For a day of thrilling 1s, head to Dufan Ancol...",-61253124,1068335377
...,...,...,...,...,...,...
3166,3,Depok Fantasi Waterpark,1,Taman wisata air dengan berbagai wahana permai...,"Jalan Raya Margonda, Depok",NaN
3167,3,Situ Pengasinan Depok,1,Danau yang menjadi tempat wisata favorit masya...,"Jalan Raya Sawangan, Bojongsari, Depok",NaN
3168,3,Cagar Alam Kota Depok,1,Cagar alam yang menjadi habitat bagi berbagai ...,"Jalan Raya Pasir Putih, Sawangan, Depok",NaN


In [5]:
train = df_user_expanded[df_user_expanded.user_split=="train"]
test = df_user_expanded[df_user_expanded.user_split=="test"]

In [6]:
train.head()

,user_interest,user_past_event,latitude,longitude,user_split,row_number
user_id,,,,,,
1,2,5,-6.333183,106.774398,train,1
1,2,1,-6.333183,106.774398,train,2
1,1,5,-6.333183,106.774398,train,3
1,1,1,-6.333183,106.774398,train,4
2,1,1,-6.274905,106.821413,train,5


In [7]:
test.head()

,user_interest,user_past_event,latitude,longitude,user_split,row_number
user_id,,,,,,
101,1,5,-6.106006,106.854229,test,401
101,1,4,-6.106006,106.854229,test,402
101,4,5,-6.106006,106.854229,test,403
101,4,4,-6.106006,106.854229,test,404
102,2,1,-6.292631,106.743851,test,405


In [8]:
print("Number of user in train:",train.shape[0])
print("Number of user in test:",test.shape[0])

Number of user in train: 400
Number of user in test: 400


In [9]:
df_event["combined"] = df_event["event_name"] + " " + df_event["event_description"]
df_event.combined


event_id
111     Jakarta Bird Land Ancol Jakarta Bird Land adal...
122     Magic Art 3D Museum Jakarta Step into a realm ...
113     Sea World Ancol Dive into the depths of the oc...
114     Samudra Ancol Experience the magic of marine l...
115     Dufan Ancol For a day of thrilling 1s, head to...
                              ...                        
3166    Depok Fantasi Waterpark Taman wisata air denga...
3167    Situ Pengasinan Depok Danau yang menjadi tempa...
3168    Cagar Alam Kota Depok Cagar alam yang menjadi ...
3169    Masjid Dian Al-Mahri Masjid megah dengan kubah...
3170    Universitas Indonesia Kampus terbesar di Indon...
Name: combined, Length: 68, dtype: object

In [10]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [12]:
df_event["combined"] = df_event["combined"].str.lower()
df_event.combined

event_id
111     jakarta bird land ancol jakarta bird land adal...
122     magic art 3d museum jakarta step into a realm ...
113     sea world ancol dive into the depths of the oc...
114     samudra ancol experience the magic of marine l...
115     dufan ancol for a day of thrilling 1s, head to...
                              ...                        
3166    depok fantasi waterpark taman wisata air denga...
3167    situ pengasinan depok danau yang menjadi tempa...
3168    cagar alam kota depok cagar alam yang menjadi ...
3169    masjid dian al-mahri masjid megah dengan kubah...
3170    universitas indonesia kampus terbesar di indon...
Name: combined, Length: 68, dtype: object

In [13]:
def tokenize_text(x):
    if pd.isna(x):
        return []

    x = re.sub(r'\d+', '', x)
    return word_tokenize(x.lower())

df_event["tokenized"] = df_event["combined"].apply(tokenize_text)

In [14]:
df_event["tokenized"]

event_id
111     [jakarta, bird, land, ancol, jakarta, bird, la...
122     [magic, art, d, museum, jakarta, step, into, a...
113     [sea, world, ancol, dive, into, the, depths, o...
114     [samudra, ancol, experience, the, magic, of, m...
115     [dufan, ancol, for, a, day, of, thrilling, s, ...
                              ...                        
3166    [depok, fantasi, waterpark, taman, wisata, air...
3167    [situ, pengasinan, depok, danau, yang, menjadi...
3168    [cagar, alam, kota, depok, cagar, alam, yang, ...
3169    [masjid, dian, al-mahri, masjid, megah, dengan...
3170    [universitas, indonesia, kampus, terbesar, di,...
Name: tokenized, Length: 68, dtype: object

In [15]:
df_event["clean_tokenized"] = df_event["tokenized"].apply(lambda tokens: [word for word in tokens if word.isalpha() and word not in stopwords.words("english")])
df_event.drop(columns=["combined","tokenized"], inplace=True)

In [16]:
glove = gensim.downloader.load('glove-wiki-gigaword-100')

In [17]:
glove["museum"]

array([ 0.28537 ,  0.75628 , -0.17795 ,  0.11613 ,  0.97585 , -0.12008 ,
        0.21697 ,  0.71994 , -0.031774, -0.27201 ,  0.024977, -0.33441 ,
        0.049251,  0.20123 , -0.54049 ,  0.55483 ,  1.3269  ,  0.6578  ,
       -0.70861 ,  0.13022 , -0.1677  ,  0.20343 , -0.3317  , -0.68989 ,
        0.25454 , -0.72728 , -0.17238 , -0.3029  ,  0.23326 , -0.60742 ,
       -1.1749  , -0.046057, -0.60587 ,  1.3074  , -0.37978 ,  0.12758 ,
       -0.051791, -0.12641 , -0.15013 , -0.72033 ,  0.43708 , -0.95698 ,
       -0.1413  , -0.11999 ,  0.43878 ,  0.027117,  0.59021 ,  0.24453 ,
        0.52182 ,  0.27636 ,  0.71995 , -0.57356 ,  0.28636 , -0.15308 ,
       -0.98853 , -1.7872  , -0.56086 , -0.66083 ,  1.6703  ,  0.34054 ,
        0.024841,  1.3854  ,  0.25459 ,  0.47657 ,  0.59289 , -0.050394,
        0.11254 ,  0.88479 ,  0.14151 ,  0.64695 ,  0.4851  ,  0.010365,
        0.025033, -0.32815 ,  0.33235 , -0.71439 ,  1.0167  ,  0.2442  ,
       -1.1874  ,  0.37168 , -0.50951 ,  0.31758 , 

In [18]:
def get_embedding(list_of_tokens):
  embeddings = np.zeros(100)
  for token in list_of_tokens:
    if token in glove:
      embeddings += glove[token]
  return embeddings

In [19]:
df_event["embedding"] = df_event["clean_tokenized"].apply(lambda x: get_embedding(x))

In [20]:
df_event["embedding"]

event_id
111     [6.384871584363282, -7.375987129285932, -0.831...
122     [3.458111945539713, 9.057472046464682, 4.98886...
113     [-1.8965998915955424, 9.62886660778895, 14.171...
114     [-1.6508010225370526, 4.214751996099949, 9.346...
115     [-2.3796889735385776, 4.200980119407177, 6.369...
                              ...                        
3166    [-1.8012420386075974, -3.306480050086975, 1.61...
3167    [1.1175260171294212, -1.876307049766183, 0.228...
3168    [-1.2199020460247993, -3.708836942911148, 2.36...
3169    [2.8056099861860275, -3.3937700390815735, -1.7...
3170    [0.33932600170373917, -2.969294011592865, -1.6...
Name: embedding, Length: 68, dtype: object

In [21]:
# for user_id = 17

u=17
items_of_user_17 = train.query("user_id==@u").user_interest.to_list()
items_of_user_17

['2', '2', '1', '1']

In [29]:
embedding_of_events_of_user = df_event.loc[df_event.category_id.isin(items_of_user_17), "embedding"]
embedding_of_events_of_user

Series([], Name: embedding, dtype: object)

In [30]:
user_index = 17
event_index = 111
embedding_of_events_of_user = df_event.loc[df_event.index.isin([items_of_user_17]), "embedding"]

In [31]:
df_event.loc[df_event.index.isin([111]), "embedding"]

event_id
111    [6.384871584363282, -7.375987129285932, -0.831...
Name: embedding, dtype: object

In [25]:
# df_event

In [26]:
profile_user = np.sum(embedding_of_events_of_user.values)
profile_user

0

In [27]:
profile_user.shape

AttributeError: ignored

In [ ]:
df_event["cosine"] = df_event.embedding.apply(lambda x: cosine_similarity(profile_user.reshape(1,100), x.reshape(1,100))).apply(lambda x: x[0][0])
df_event[["event_name","cosine"]].sort_values("cosine", ascending=False)